<a href="https://colab.research.google.com/github/Krishna4123/amdl/blob/main/handwrittennum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [3]:
torch.manual_seed(42)
batch_size = 64

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:

train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [6]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)

model = DigitClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train():
    model.train()
    for epoch in range(5):   # 5 epochs is enough for FC network
        total_loss = 0
        for imgs, labels in train_loader:
            preds = model(imgs)
            loss = loss_fn(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} | Loss = {total_loss/len(train_loader):.4f}")

In [8]:
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            preds = model(imgs)
            _, predicted = preds.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct/total:.2f}%")

train()
test()

Epoch 1 | Loss = 0.3375
Epoch 2 | Loss = 0.1480
Epoch 3 | Loss = 0.1120
Epoch 4 | Loss = 0.0889
Epoch 5 | Loss = 0.0775
Test Accuracy: 96.97%


In [10]:
import torch
from torchvision import transforms
from PIL import Image

# ----------- 1. Preprocessing -----------
transform = transforms.Compose([
    transforms.Grayscale(),               # Convert to 1 channel
    transforms.Resize((28, 28)),          # Resize to MNIST size
    transforms.ToTensor(),                # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Same normalization used in training
])

def predict_image(img_path):
    # Load image
    img = Image.open(img_path)

    # Apply preprocessing
    img = transform(img)
    img = img.view(1, 28*28)   # Flatten for fully connected network

    # ----------- 2. Model prediction -----------
    model.eval()
    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output, 1)

    return predicted.item()


# -------- Example Usage --------
result = predict_image("12.jpeg")
print("Predicted Digit:", result)


Predicted Digit: 2
